## Work bench to improve parsing and updating laws
This notebook is targeted to "d18-3823.pdf" and the change to the Berliner Straßengesetz.

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lawinprogress.apply_changes.apply_changes import apply_changes
from lawinprogress.parsing.change_law_utils import preprocess_raw_law
from lawinprogress.parsing.parse_change_law import (
    parse_change_request_line,
    parse_change_law_tree,
)
from lawinprogress.parsing.parse_source_law import parse_source_law_tree
from lawinprogress.parsing.lawtree import LawTextNode
from lawinprogress.parsing.proposal_pdf_to_artikles import (
    extract_law_titles,
    extract_raw_proposal,
    extract_seperate_change_proposals,
    read_pdf_law,
    remove_inkrafttreten,
)

### Read and process the change law

In [3]:
change_law_path = "../data/0145-21.pdf"  # "../data/0483-21.pdf"  # "../data/d18-3823.pdf"

In [4]:
change_law_raw_pages = read_pdf_law(change_law_path)

# idenfify the different laws affected
change_law_extract = extract_raw_proposal(change_law_raw_pages)
proposals_list = extract_seperate_change_proposals(change_law_extract)
law_titles = extract_law_titles(proposals_list)
law_titles, proposals_list = remove_inkrafttreten(law_titles, proposals_list)

In [5]:
law_titles

['Zivilprozessordnung',
 'Strafprozessordnung',
 'Elektronischer-Rechtsverkehr-Verordnung',
 'Arbeitsgerichtsgesetzes zum 1. Januar  2026',
 'Finanzgerichtsordnung zum 1. Januar 2022']

In [27]:
# we only work with the first one here
change_law = proposals_list[2]
# << do some dirty fix here, since someone forgot to add '' to the change law...
change_law = "Stelle abgewickelt werden.“".join(
    change_law.split("Stelle abgewickelt werden.")
)
print(change_law)

Änderung der Elektronischer-Rechtsverkehr-Verordnung 
Die  Elektronischer-Rechtsverkehr-Verordnung  vom  24.  November  2017  (BGBl. I 
S. 3803), die durch Artikel 1 der Verordnung vom 9. Februar 2018 (BGBl. I S. 200) geändert
worden ist, wird wie folgt geändert:
1. In § 1 Absatz 1 Satz 2 wird die Angabe „Kapitels 4“ durch „Kapitels 5“ ersetzt.
2. § 2 wird wie folgt geändert:
a) In Absatz 1 Satz 1 werden die Wörter „in druckbarer, kopierbarer und, soweit tech-
nisch möglich, durchsuchbarer Form“ gestrichen.
b) Absatz 2 wird wie folgt gefasst:
„(2) Das elektronische Dokument soll den nach § 5 Absatz 1 Nummer 1 und 6 
bekanntgemachten technischen Standards entsprechen.“ 
3. § 5 wird wie folgt geändert:
a) In der Überschrift wird das Wort „Anforderungen“ durch das Wort „Standards“ er-
setzt.
b) Absatz 1 wird wie folgt geändert:
aa) In dem Satzteil vor Nummer 1 werden die Wörter „Anforderungen an“ durch
die Wörter „Standards für“ ersetzt und werden vor dem Wort „Bearbeitung“ die 
Wörter „E

In [28]:
# parse source law
clean_change_law = preprocess_raw_law(change_law)
parsed_change_law_tree = LawTextNode(text=law_titles[0], bulletpoint="change")
parsed_change_law_tree = parse_change_law_tree(
    text=clean_change_law, source_node=parsed_change_law_tree
)

In [29]:
print(clean_change_law)

1. In § 1 Absatz 1 Satz 2 wird die Angabe „Kapitels 4“ durch „Kapitels 5“ ersetzt.
2. § 2 wird wie folgt geändert:
a) In Absatz 1 Satz 1 werden die Wörter „in druckbarer, kopierbarer und, soweit technisch möglich, durchsuchbarer Form“ gestrichen.
b) Absatz 2 wird wie folgt gefasst:„(2) Das elektronische Dokument soll den nach § 5 Absatz 1 Nummer 1 und 6 bekanntgemachten technischen Standards entsprechen.“
3. § 5 wird wie folgt geändert:
a) In der Überschrift wird das Wort „Anforderungen“ durch das Wort „Standards“ ersetzt.
b) Absatz 1 wird wie folgt geändert:
aa) In dem Satzteil vor Nummer 1 werden die Wörter „Anforderungen an“ durchdie Wörter „Standards für“ ersetzt und werden vor dem Wort „Bearbeitung“ dieWörter „Eignung zur“ eingefügt.
bb) In Nummer 5 wird der Punkt am Ende durch das Wort „und“ ersetzt.
cc) Folgende Nummer 6 wird angefügt:„6.  die technischen Eigenschaften der elektronischen Dokumente.“
c) In Absatz 2 werden jeweils die Wörter „Anforderungen“ durch das Wort „Standar

In [30]:
parsed_change_law_tree._print()

change - Zivilprozessordnung
├── 1. - In § 1 Absatz 1 Satz 2 wird die Angabe „Kapitels 4“ durch „Kapitels 5“ ersetzt.
├── 2. - § 2 wird wie folgt geändert:
│   ├── a) - In Absatz 1 Satz 1 werden die Wörter „in druckbarer, kopierbarer und, soweit technisch möglich, durchsuchbarer Form“ gestrichen.
│   └── b) - Absatz 2 wird wie folgt gefasst:„(2) Das elektronische Dokument soll den nach § 5 Absatz 1 Nummer 1 und 6 bekanntgemachten technischen Standards entsprechen.“
├── 3. - § 5 wird wie folgt geändert:
│   ├── a) - In der Überschrift wird das Wort „Anforderungen“ durch das Wort „Standards“ ersetzt.
│   ├── b) - Absatz 1 wird wie folgt geändert:
│   │   ├── aa) - In dem Satzteil vor Nummer 1 werden die Wörter „Anforderungen an“ durchdie Wörter „Standards für“ ersetzt und werden vor dem Wort „Bearbeitung“ dieWörter „Eignung zur“ eingefügt.
│   │   ├── bb) - In Nummer 5 wird der Punkt am Ende durch das Wort „und“ ersetzt.
│   │   └── cc) - Folgende Nummer 6 wird angefügt:„6.  die technisc

In [31]:
# build up a list of texts that describe a single change
all_change_lines = []
for l in parsed_change_law_tree.leaves:
    path = [str(l)]
    node = l
    while node.parent:
        node = node.parent
        path.append(str(node))
    change_line = " ".join(path[::-1][1:])
    all_change_lines.append(change_line)

In [32]:
# parse the change requests in a structured format
change_requests = []
for change_request_line in all_change_lines:
    res = parse_change_request_line(change_request_line)
    if res:
        change_requests.extend(res)

In [33]:
all_change_lines[-2]

'8. - Der bisherige § 11 wird § 15.'

In [34]:
change_requests

[Change(location=['§ 1', 'Absatz 1'], sentences=['Satz 2'], text=['Kapitels 4', 'Kapitels 5'], change_type='replace'),
 Change(location=['§ 2', 'Absatz 1'], sentences=['Satz 1'], text=['in druckbarer, kopierbarer und, soweit technisch möglich, durchsuchbarer Form'], change_type='delete_after'),
 Change(location=['§ 2', 'Absatz 2'], sentences=[], text=['(2) Das elektronische Dokument soll den nach § 5 Absatz 1 Nummer 1 und 6 bekanntgemachten technischen Standards entsprechen.'], change_type='rephrase'),
 Change(location=['§ 5', 'Überschrift'], sentences=[], text=['Anforderungen', 'Standards'], change_type='replace'),
 Change(location=['§ 5', 'Absatz 1', 'Nummer 1'], sentences=[], text=['Anforderungen an', 'Standards für', 'Bearbeitung', 'Eignung zur'], change_type='MULTIPLE_CHANGES'),
 Change(location=['§ 5', 'Absatz 1', 'Nummer 5'], sentences=[], text=['und'], change_type='replace'),
 Change(location=['§ 5', 'Absatz 1', 'Nummer 6'], sentences=[], text=['6.  die technischen Eigenschafte

### Process the source law

In [35]:
law_name = "Elektronischer-Rechtsverkehr-Verordnung"  # "Berliner Straßengesetz"
source_law_path = "../data/source_laws/{}.txt".format(law_name)

In [36]:
# read the source law
with open(source_law_path, "r") as file:
    source_law_text = file.read()

In [43]:
# parse source law
parsed_law_tree = LawTextNode(text=law_name, bulletpoint="source")
parsed_law_tree = parse_source_law_tree(
    text=source_law_text, source_node=parsed_law_tree
)

In [44]:
parsed_law_tree._print()

source - Elektronischer-Rechtsverkehr-Verordnung
├── § 1 - Anwendungsbereich
│   ├── (1) - Diese Verordnung gilt für die Übermittlung elektronischer Dokumente an die Gerichte der Länder und des Bundes sowie die Bearbeitung elektronischer Dokumente durch diese Gerichte nach § 130a der Zivilprozessordnung, § 46c des Arbeitsgerichtsgesetzes, § 65a des Sozialgerichtsgesetzes, § 55a der Verwaltungsgerichtsordnung und § 52a der Finanzgerichtsordnung. Sie gilt ferner nach Maßgabe des Kapitels 4 für die Übermittlung elektronischer Dokumente an Strafverfolgungsbehörden und Strafgerichte der Länder und des Bundes nach § 32a der Strafprozessordnung sowie die Bearbeitung elektronischer Dokumente.
│   └── (2) - Besondere bundesrechtliche Vorschriften über die Übermittlung elektronischer Dokumente und strukturierter maschinenlesbarer Datensätze bleiben unberührt.
Kapitel 2
Technische Rahmenbedingungen des elektronischen Rechtsverkehrs
Nichtamtliches Inhaltsverzeichnis
├── § 2 - Anforderungen an elek

In [45]:
print(parsed_law_tree.to_text())

source Elektronischer-Rechtsverkehr-Verordnung
    § 1 Anwendungsbereich
        (1) Diese Verordnung gilt für die Übermittlung elektronischer Dokumente an die Gerichte der Länder und des Bundes sowie die Bearbeitung elektronischer Dokumente durch diese Gerichte nach § 130a der Zivilprozessordnung, § 46c des Arbeitsgerichtsgesetzes, § 65a des Sozialgerichtsgesetzes, § 55a der Verwaltungsgerichtsordnung und § 52a der Finanzgerichtsordnung. Sie gilt ferner nach Maßgabe des Kapitels 4 für die Übermittlung elektronischer Dokumente an Strafverfolgungsbehörden und Strafgerichte der Länder und des Bundes nach § 32a der Strafprozessordnung sowie die Bearbeitung elektronischer Dokumente.
        (2) Besondere bundesrechtliche Vorschriften über die Übermittlung elektronischer Dokumente und strukturierter maschinenlesbarer Datensätze bleiben unberührt.
Kapitel 2
Technische Rahmenbedingungen des elektronischen Rechtsverkehrs
Nichtamtliches Inhaltsverzeichnis
    § 2 Anforderungen an elektronische 

### Update the source law

In [46]:
res_law_tree = apply_changes(parsed_law_tree, change_requests)

APPLIED replace: Change(location=['§ 1', 'Absatz 1'], sentences=['Satz 2'], text=['Kapitels 4', 'Kapitels 5'], change_type='replace')
APPLIED delete_after: Change(location=['§ 2', 'Absatz 1'], sentences=['Satz 1'], text=['in druckbarer, kopierbarer und, soweit technisch möglich, durchsuchbarer Form'], change_type='delete_after')
APPLIED rephrase: Change(location=['§ 2', 'Absatz 2'], sentences=[], text=['(2) Das elektronische Dokument soll den nach § 5 Absatz 1 Nummer 1 und 6 bekanntgemachten technischen Standards entsprechen.'], change_type='rephrase')
Location Überschrift not found.
No path found in Change(location=['§ 5', 'Überschrift'], sentences=[], text=['Anforderungen', 'Standards'], change_type='replace'). SKIPPING
SKIPPED MULTIPLE_CHANGES: Change(location=['§ 5', 'Absatz 1', 'Nummer 1'], sentences=[], text=['Anforderungen an', 'Standards für', 'Bearbeitung', 'Eignung zur'], change_type='MULTIPLE_CHANGES')
not enougth text to replace
WITHOUT CHANGE replace: Change(location=['§ 5

AttributeError: 'NoneType' object has no attribute 'remove_child'

In [41]:
print(res_law_tree.to_text())

NameError: name 'res_law_tree' is not defined